In [36]:
import datetime
import requests
import time
import json
import os

In [37]:
LOGS_DIR = "../../data/perf/logs"
NANOSECONDS_IN_SECOND = 1e9
LIMIT = 1000

In [44]:
base_url = "http://le-terror:3100/loki/api/v1/query_range"
query = '{compose_service="script"}'

In [39]:
time_chunks = [
    (
        "meilisearch_indexing",
        "2025-02-26T20:30:00+01:00",
        "2025-02-27T15:50:00+01:00"
    )
]


In [51]:
for time_chunk in time_chunks:
    label, chunk_start, chunk_end = time_chunk

    start_parsed = datetime.datetime.fromisoformat(chunk_start)
    end_parsed = datetime.datetime.fromisoformat(chunk_end)

    start = int(start_parsed.timestamp() * NANOSECONDS_IN_SECOND)
    end = int(end_parsed.timestamp() * NANOSECONDS_IN_SECOND)

    accumulator = []
    cursor = start

    while cursor < end:
        params = {
            "query": query,
            "start": cursor,
            "end": end,
            "limit": LIMIT,
            "direction": "forward"
        }

        response = requests.get(base_url, params=params)
        response_json = response.json()

        data = []
        for container in response_json["data"]["result"]:
            data.extend(container["values"])

        accumulator.extend(data)

        if len(data) < LIMIT:
            break

        cursor = int(data[-1][0]) + 1

    with open(f"{LOGS_DIR}/{label}.json", "w") as file:
        json.dump(accumulator, file, indent=2)